In [21]:
import pandas as pd
import numpy as np
from copy import deepcopy

In [22]:
data1 = pd.read_csv('CA2Data/animals', header=None, delimiter=' ')
data2 = pd.read_csv('CA2Data/countries', header=None, delimiter=' ')
data3 = pd.read_csv('CA2Data/fruits', header=None, delimiter=' ')
data4 = pd.read_csv('CA2Data/veggies', header=None, delimiter=' ')
# dataset = pd.concat([data1, data2, data3, data4])
# print(data1.shape, data2.shape, data3.shape, data4.shape)
dataset = pd.concat([data1.iloc[:,1:], data2.iloc[:,1:], data3.iloc[:,1:], data4.iloc[:,1:]])

In [23]:
dataset

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,-0.015926,-0.079864,-0.332180,0.034246,0.11045,0.060519,-1.18300,-0.709950,-0.153610,-0.17446,...,-0.13267,0.467350,0.086454,0.244950,-0.621270,0.377990,0.709060,-0.023002,0.003908,-0.035713
1,0.477270,-0.915870,-0.297700,-0.224890,0.55337,0.177720,-1.56030,-0.405800,-0.375230,-0.30488,...,-0.10108,0.351080,0.312520,0.162300,0.022012,0.048974,-0.027862,-0.446990,-0.249570,0.028510
2,-0.335750,0.388970,-0.419290,-0.332190,0.53170,-0.258390,-2.38690,-0.434430,-0.397600,-0.99356,...,-0.11501,-0.558040,0.484270,0.251670,-0.068426,0.342270,0.268400,-0.149290,-0.235160,0.039194
3,0.211100,0.217630,-0.526380,-0.422770,0.84672,-0.038383,-1.92010,-0.309590,0.021942,-0.49525,...,-0.15757,0.131710,0.632910,-0.321110,-0.046260,-0.179710,-0.001380,-0.150760,-0.240640,-0.051365
4,0.081110,-0.502850,-0.055975,0.459650,-0.30271,0.066912,0.13084,-0.404260,-0.290590,-0.08072,...,0.15573,0.043208,-0.475560,-0.469140,-0.131460,0.285380,-0.051485,-0.400790,0.231430,0.229130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,-0.844340,-0.259140,-0.085688,-0.687480,0.42423,-0.699730,-0.92894,-0.025033,0.429040,-1.38160,...,-0.39075,-0.124110,0.187310,0.391210,0.173610,-0.151170,-0.227640,-0.099521,-0.360730,0.326480
54,-0.430230,-0.409770,0.230610,0.242560,0.52562,-0.988400,-1.31670,-0.107950,-0.268680,-0.91291,...,-0.27758,-0.802990,0.515360,0.411270,0.299350,0.046180,-0.196720,-0.463350,-0.562020,-0.358750
55,-0.362160,-0.538600,-0.660520,0.359790,0.22559,0.009144,0.50548,-0.509020,0.228950,-0.12552,...,0.42165,-0.186220,-0.343260,0.144860,-0.782220,-0.163550,-0.143750,0.026223,0.140610,-0.385060
56,-0.143230,-0.317580,-0.391740,-0.637460,-0.12007,-0.285030,0.33173,-0.067649,-0.053089,-0.18279,...,-0.29524,-0.228070,-0.333430,0.219130,-0.289700,-0.290250,0.311860,-0.562860,-0.589640,0.252690


In [24]:
dataset = np.array(dataset)
dataset.shape
len(dataset)

327

In [ ]:
k_choices = np.arange(1,10)
k_choices

In [25]:
def distance(X,Y):
    #Return the Euclidean distance between X and Y
    return np.linalg.norm(X-Y)

In [ ]:
# k = 4

# centroids_dict = {}
# for i in range(k):
#     centroids_dict.update({i: centroids[i]})
# # print(centroids_dict)

# clusters_dict = {}
# for i in range(k):
#     clusters_dict.update({i: []})
# # print(clusters_dict)


In [13]:
def check_converge():
    pass

In [32]:

k=4
num_centroids = k

centroids = dataset[np.random.randint(dataset.shape[0], size=num_centroids), :]
# print(centroids.shape)
# print(centroids)


def get_centroid_positions():
    data_centroid_indices = np.zeros((len(dataset),1))

    num_datapoints = len(dataset)
    for index in range(num_datapoints):
        distances = np.zeros((num_centroids, 1))
        for centroid_index in range(num_centroids):
            distance_to_centroid = distance(dataset[index, :], centroids[centroid_index, :])
            distances[centroid_index] = distance_to_centroid
        closest_centroid_position = np.argmin(distances)
        data_centroid_indices[index] = closest_centroid_position
#     print(data_centroid_indices)
    return data_centroid_indices


# update centroids
for _ in range(1000):
    num_centroids = k
    data_centroid_indices = get_centroid_positions()
#     print(data_centroid_indices)
    data = deepcopy(dataset)
    
    # data[np.int8(data_centroid_indices) == [0]]
    for centroid_index in range(num_centroids):
        centroids[centroid_index] = np.mean(data[data_centroid_indices.flatten() == centroid_index], axis=0)
print(centroids)


[[-0.07048999 -0.20060509 -0.26966388 ... -0.1941668   0.06881185
   0.18472587]
 [-0.1049841  -0.11144429 -0.06780627 ... -0.08930704 -0.22906217
   0.13883341]
 [ 0.38400932  0.31275504  0.06034656 ...  0.08786514  0.11787716
   0.25177092]
 [-0.24321728 -0.0922127   0.0852466  ... -0.44217913 -0.06488467
   0.05516707]]


In [29]:
d = np.array([[1,2,3], 
              [4,5,6]])
print(d)
np.mean(d)

[[1 2 3]
 [4 5 6]]


3.5